In [66]:
# Define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'
MODEL_INPUT = MODELS_DIR + 'input.cc'
MODEL_OUTPUT = MODELS_DIR + 'output.cc'
INPUT_DATA_NPY= MODELS_DIR + 'input_data.npy'
OUTPUT_DATA_NPY= MODELS_DIR + 'output_data.npy'
!pip3 install tensorflow

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable


In [39]:
# TensorFlow is an open source machine learning library
import tensorflow as tf

# Keras is TensorFlow's high-level API for deep learning
# Numpy is a math library
import numpy as np
# Pandas is a data manipulation library 
import pandas as pd
# Matplotlib is a graphing library
import matplotlib.pyplot as plt
# Math is Python's math library
import math
from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.datasets import cifar10

import numpy as np

# Set seed for experiment reproducibility
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

# 1. Design the model

In [40]:

#all_cnn_net
# https://github.com/AnuragGorkar/Image_Recognition_using_AllCNN_architechture/blob/master/Image_Recognition_using_AllCNN_architechture.ipynb
model = Sequential()

model.add(Conv2D(96, (3, 3), padding = "same", input_shape = (32, 32, 3))) #----> 1 #n,h,w,c
model.add(Activation('relu')) #----> 1

    # INTERMEDIATE MODEL LAYERS
model.add(Conv2D(96, (3, 3), padding = "same")) #----> 2
model.add(Activation('relu')) #----> 2

model.add(Conv2D(96, (3, 3), padding = "same", strides = (2,2))) #----> 3
model.add(Dropout(0.5))

model.add(Conv2D(192, (3,3), padding = "same")) #----> 4
model.add(Activation('relu')) #----> 4

model.add(Conv2D(192, (3,3), padding = "same")) #----> 5
model.add(Activation('relu')) #----> 5

model.add(Conv2D(192, (3,3), padding = "same", strides = (2,2))) #----> 6
model.add(Dropout(0.5)) #----> 6

model.add(Conv2D(192, (3,3),padding = "same")) #----> 7
model.add(Activation('relu')) #----> 7

model.add(Conv2D(192, (1,1),padding = "valid")) #----> 8
model.add(Activation('relu')) #----> 8

model.add(Conv2D(10, (1,1),padding = "valid")) #----> 9
model.add(Activation('relu')) #----> 9

# ADD GLOBAL AVERAGE POOLING LAYER WITH SOFTMAX
model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

print (model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
activation_47 (Activation)   (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
activation_48 (Activation)   (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 16, 16, 96)        83040     
_________________________________________________________________
dropout_12 (Dropout)         (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 16, 16, 192)      

In [61]:
model_select = Sequential()

model_select.add(Conv2D(96, (3, 3), padding = "same", input_shape = (32, 32, 3))) #----> 1 #n,h,w,c
model_select.add(Activation('relu')) #----> 1

print (model_select.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_63 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
activation_55 (Activation)   (None, 32, 32, 96)        0         
Total params: 2,688
Trainable params: 2,688
Non-trainable params: 0
_________________________________________________________________
None


# 2. Train Model (skip now)

In [41]:
#(x_train, y_train),(x_test, y_test) = cifar10.load_data()

In [62]:
model_select.save(MODEL_TF)

INFO:tensorflow:Assets written to: models/model/assets


INFO:tensorflow:Assets written to: models/model/assets


# 3. Generate a TensorFlow Lite Model


## 1. Generate Models with or without Quantization
We now have an acceptably accurate model. We'll use the TensorFlow Lite Converter to convert the model into a special, space-efficient format for use on memory-constrained devices.

Since this model is going to be deployed on a microcontroller, we want it to be as tiny as possible! One technique for reducing the size of a model is called quantization. It reduces the precision of the model's weights, and possibly the activations (output of each layer) as well, which saves memory, often without much impact on accuracy. Quantized models also run faster, since the calculations required are simpler.

In the following cell, we'll convert the model twice: once with quantization, once without

In [63]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

# Convert the model to the TensorFlow Lite format with quantization
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 32, 32, 3)
      yield [data.astype(np.float32)]
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)

7264

# Generate a TensorFlow Lite for Microcontrollers Model
Convert the TensorFlow Lite quantized model into a C source file that can be loaded by TensorFlow Lite for Microcontrollers.

In [64]:
# Install xxd if it is not available
#!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

In [84]:
# Reference: https://www.tensorflow.org/lite/performance/post_training_quantization
interpreter = tf.lite.Interpreter(model_path=str(MODEL_TFLITE))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

input_data = np.random.rand(1, 32, 32, 3).astype(np.int8)
# Check if the input type is quantized, then rescale input data to uint8
if input_details['dtype'] == np.uint8:
    input_scale, input_zero_point = input_details["quantization"]
    input_data = input_data / input_scale + input_zero_point

interpreter.set_tensor(input_details["index"], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details["index"])[0]

In [86]:
# save input and expected output
np.save(INPUT_DATA_NPY, input_data)
np.save(OUTPUT_DATA_NPY, input_data)
!xxd -i {INPUT_DATA_NPY} > {MODEL_INPUT}
!xxd -i {OUTPUT_DATA_NPY} > {MODEL_OUTPUT}